# Рекомендация тарифов

## Описание проекта
В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Описание данных
Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:
* сalls — количество звонков,
* minutes — суммарная длительность звонков в минутах,
* messages — количество sms-сообщений,
* mb_used — израсходованный интернет-трафик в Мб,
* is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

## Откроем и изучим файл

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression 
import warnings
warnings.filterwarnings("ignore")

df = pd.read_csv('/datasets/users_behavior.csv')

df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Импортировали нужные библиотеки, открыли файл с данными и изучили его: пропусков действительно нет, целевой признак - `is_ultra`, решаем задачу классификации.

## Поделим данные на выборки

Объявим переменные для признаков

In [3]:
features = df.drop(['is_ultra'], axis=1) 
target = df['is_ultra'] 

Разделим данные на три части:
60% оставим для тренировочной;
40% поровну для валидационной и тестовой. 

In [4]:
features_train, features_other, target_train, target_other = train_test_split(
features, target, test_size=0.4, random_state=12345) 

features_valid, features_test, target_valid, target_test = train_test_split(
features_other, target_other, test_size=0.5, random_state=12345)

## Исследуем модели

### Дерево решений


In [5]:
best_model = None
best_result = 0
for depth in range(1, 6):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth) 
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    result = model.score(features_valid, target_valid)
    if result > best_result:
        best_model = model
        best_result = result
        
print('Accuracy наилучшей модели на валидационной выборке:', best_result)
print(best_model)

Accuracy наилучшей модели на валидационной выборке: 0.7853810264385692
DecisionTreeClassifier(max_depth=3, random_state=12345)


Accuracy впечатляет. Но в спринте говорится что дерево решений не лидирует по качеству.

### Случайный лес

In [6]:
best_model = None
best_result = 0
for est in range(10, 200, 10):
    model = RandomForestClassifier(random_state=12345, n_estimators=est)
    model.fit(features_train, target_train)
    result = model.score(features_valid, target_valid)
    if result > best_result:
        best_model = model
        best_result = result

print('Accuracy наилучшей модели на валидационной выборке:', best_result)
print(best_model)

Accuracy наилучшей модели на валидационной выборке: 0.7916018662519441
RandomForestClassifier(n_estimators=50, random_state=12345)


Accuracy 0.79 при количестве деревьев 23. Зафиксируем это значение и подберем глубину:

In [7]:
best_model = None
best_result = 0
for depth in range(1, 7):
    model = RandomForestClassifier(random_state=12345, n_estimators=23, max_depth=depth)
    model.fit(features_train, target_train)
    result = model.score(features_valid, target_valid)
    if result > best_result:
        best_model = model
        best_result = result

print('Accuracy наилучшей модели на валидационной выборке:', best_result)
print(best_model)

Accuracy наилучшей модели на валидационной выборке: 0.8009331259720062
RandomForestClassifier(max_depth=6, n_estimators=23, random_state=12345)


Accuracy стал выше при глубине = 6.

In [8]:
best_model = None
best_result = 0
for rand in range(1, 400):
    model = RandomForestClassifier(random_state=rand, n_estimators=23, max_depth=6)
    model.fit(features_train, target_train)
    result = model.score(features_valid, target_valid)
    if result > best_result:
        best_model = model
        best_result = result

print('Accuracy наилучшей модели на валидационной выборке:', best_result)
print(best_model)

Accuracy наилучшей модели на валидационной выборке: 0.8102643856920684
RandomForestClassifier(max_depth=6, n_estimators=23, random_state=392)


### Логистическая регрессия

In [9]:
model = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000)
model.fit(features_train, target_train) 
model.predict(features_valid) 
print('Accuracy на валидационной выборке:', model.score(features_valid, target_valid))

Accuracy на валидационной выборке: 0.7107309486780715


Исходя из исследованных моделей логистическая регрессия занимает последнее место по параметру качества. Далее дерево решений и лидер по качеству - Случайный лес.


## Проверим модель на тестовой выборке

In [10]:
model = RandomForestClassifier(random_state=12345, n_estimators=23, max_depth=6)
model = model.fit(features_train, target_train)
predictions_test = model.predict(features_test)
print('Accuracy:', accuracy_score(target_test, predictions_test))


Accuracy: 0.7947122861586314


## (бонус) Проверим модели на адекватность

In [11]:
from sklearn.dummy import DummyClassifier

dclf = DummyClassifier(strategy = 'most_frequent', random_state = 12345)
dclf.fit(features_train, target_train)
score = dclf.score(features_valid, target_valid)
print(score)

0.7060653188180405


Получается любая модель лучше чем эта. Разве что логистическая регрессия недалеко ушла.

## Вывод

В данном проекте построили модель с достаточным значением accuracy. Проверка на тестовой выборке показала Accuracy: 0.79. 